## Lab: Superdense Coding

Superdense coding is a quantum communication protocol in which two classical bits can be sent by transmitting only one qubit, provided the communicating parties share an entangled pair. Alice encodes her message by applying one of four possible operations to her qubit before sending it to Bob. Bob then performs a joint measurement in the Bell basis on both qubits, which directly produces the two-bit message. This enables the transmission of two classical bits using just a single qubit when prior entanglement is available.

### Task

Alice intends to send an **8-bit** random key to Bob using the superdense coding protocol. The task is to create a circuit that tests this protocol.

### Expected Output

The program should draw the circuit for each bit-pair, and after the transmission is complete display the full key Alice sent, the key Bob received, and report whether the transmission was successful or failed.

### Optional Challenge

Modify the code so that it uses one circuit with 8 qubit lines to transfer the entire 8-bit key in parallel.


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import random

def bell_pair(qc, a, b):
    """Create a Bell pair between qubits a (Alice) and b (Bob)."""
    qc.h(a)
    qc.cx(a, b)
    return qc

def encode(qc, a, b0, b1):
    """Alice encodes her two classical bits onto her qubit."""
    qc.barrier()
    if (b0, b1) == (0, 1):
        qc.x(a)
    elif (b0, b1) == (1, 0):
        qc.z(a)
    elif (b0, b1) == (1, 1):
        qc.x(a)
        qc.z(a)
    qc.barrier()
    return qc

def decode(qc, a, b):
    """Bob decodes to recover Alice's two classical bits."""
    qc.cx(a, b)
    qc.h(a)
    return qc

# ------------------------------------------------
#                main program
# ------------------------------------------------

# --- Alice creates an 8-bit random key ---
alice_bits = [random.randint(0, 1) for _ in range(8)]

# --- Split key into pairs for superdense coding ---
bit_pairs = []
for i in range(0, len(alice_bits), 2):
    first_bit = alice_bits[i]
    second_bit = alice_bits[i + 1]
    bit_pairs.append((first_bit, second_bit))

# --- Simulate sending the key ---
simulator = AerSimulator()
bob_bits = []

for b0, b1 in bit_pairs:
    qc = QuantumCircuit(2, 2)  # 2 qubits (Alice, Bob), 2 classical bits
    
    # Share entanglement
    bell_pair(qc, 0, 1)
    
    # Alice encodes her 2 bits
    encode(qc, 0, b0, b1)
    
    # Bob decodes
    decode(qc, 0, 1)
    
    # Measure both qubits
    qc.measure([0, 1], [0, 1])
    print(f"Circuit for bit pair {b0}{b1}:")
    display(circuit_drawer(qc, output="mpl"))
    
    # Run the circuit
    result = simulator.run(qc, shots=1, memory=True).result()
    mem = result.get_memory()[0]  # e.g., '10' (c1c0)
    
    # Extract bits in correct order (first_bit = c0, second_bit = c1)
    c1, c0 = int(mem[0]), int(mem[1])
    bob_bits.extend([c0, c1])

# --- Show results ---
print("Alice's key:    ", "".join(map(str, alice_bits)))
print("Bob's key:      ", "".join(map(str, bob_bits)))

if bob_bits == alice_bits:
    print("Transmission OK: keys match.")
else:
    print("Transmission FAILED: keys differ.")